In [2]:
# do pip installs to get libraries: wordfreq, googletrans, editdistance, unidecode

from wordfreq import word_frequency
from googletrans import Translator
import editdistance
from unidecode import unidecode

import time

translator = Translator()

This script will process the data files to add information about translations and frequencies.

In [3]:
lang='fr_en'
fn = 'data/data_{0}/{0}.slam.20171218.train.new'.format(lang)
(src, dest)=lang.split('_')
print("Translating %s to %s" %(src,dest))


Translating fr to en


In [4]:
fn

'data/data_fr_en/fr_en.slam.20171218.train.new'

In [5]:
#Todd's code
# trans_cache = {}

# newlines=[]
# with open(fn, 'r') as f:
#     for line in f:
#         if len(line)>1:
#             #print("***"+line)
#             if line[0]!='#':
#                 # process and translate
#                 pieces = line.split()
#                 idx = pieces[0]
#                 word = pieces[1]
#                 root_token = pieces[2]
#                 part_of_speech = pieces[3]
#                 morph_feats = pieces[4]
#                 dependency_label = pieces[5]
#                 dependency_edge_head = pieces[6]
                
#                 clean_word = word.lower()
#                 if '\'' not in clean_word:
#                     if clean_word not in trans_cache:
#                         translation_obj = translator.translate(clean_word,src=src,dest=dest)
#                         translation = translation_obj.text
#                         print("%s to %s" % (clean_word, translation))  # need to do some clean up of the translation
#                         trans_cache[clean_word]=translation
#                     else:
#                         translation = trans_cache[clean_word]
                
#                 trans_freq = word_frequency(translation, dest)
#                 src_frec = word_frequency(word, src)
                
                
#                 if len(pieces) == 8:
#                     label = pieces[7]
#                     repl = [idx, word, root_token, part_of_speech, morph_feats, dependency_label, dependency_edge_head, label]
#                 else:
#                     repl = [idx, word, root_token, part_of_speech, morph_feats, dependency_label, dependency_edge_head]
#                 newlines.append("\t".join(repl)+'\n')
#             else:
#                 newlines.append(line)
#         else:
#             newlines.append(line)

# print(len(newlines))
# with open('test.txt','w') as fp:
#     fp.writelines(newlines)

In [7]:
import pandas as pd
import numpy as np

In [29]:
aoa_xl = pd.ExcelFile("data/13428_2013_348_MOESM1_ESM.xlsx")
aoa_df = aoa_xl.parse('Sheet1')

In [36]:
#Pam's code
# FOR EVERY ROOT WORD (e.g, "be" instead of "is")
# Make csv of translation and calculate a simple metric of how much of a "cognate" it is

#output filename
fnout = "{}_{}_rootwordfeats.txt".format(src,dest)
print("Will save translations and features to: ",fnout)


trans_cache = {}

newlines=[]

with open(fn, 'r') as f:
    for line in f:
        if len(line)>1 and line[0]!='#':
            # process and translate
            pieces = line.split()
            idx = pieces[0]
            word = pieces[1]
            root_token = pieces[2]
            part_of_speech = pieces[3]
            morph_feats = pieces[4]
            dependency_label = pieces[5]
            dependency_edge_head = pieces[6]

            clean_word = root_token.lower() # Only using root token here rather than word
            if '\'' not in clean_word and clean_word not in trans_cache:
                # Wait so as not to annoy the API (otherwise will give JSON error)
                time.sleep(0.2)

                # Translate so we know the word in the language the user already knows
                translation_obj = translator.translate(clean_word,src=src,dest=dest)
                translation = translation_obj.text

                # Clean up translation: 
                # remove accents, make lowercase, and remove any non-informative first words
                clean_trans = unidecode(translation)
                clean_trans = clean_trans.lower() 
                g = clean_trans.split(" ")
                if len(g) == 2:
                    if g[0] in ['to','i','we','you','they','he','she','the',"i'll"]:
                        clean_trans = g[1]          
                elif len(g) > 2:
                    if g[1] in ['will']:
                        clean_trans = ' '.join(g[2:len(g)])

                # levenshtein distance features
                levdist = editdistance.eval(clean_word, clean_trans)
                levdistfrac = levdist/max(len(clean_word),len(clean_trans))
                
                # print to console translation and LD 
                print("%s to %s (LD=%i, LDfrac=%.2f)" % (clean_word, clean_trans, levdist, levdistfrac))
                trans_cache[clean_word]=translation
                
                if dest == 'en':
                    aoa_word = clean_trans
                else:
                    aoa_word = clean_word
                word_df = aoa_df[aoa_df['Word'] == aoa_word]
                if not word_df.empty:
                    aoa = word_df['Rating.Mean'].values[0]
                else:
                    aoa = np.nan

                # word frequency of translation (known language) and src words
                trans_freq = word_frequency(translation, dest)
                src_freq = word_frequency(root_token, src)

                # write to csv file
                newrow = [root_token, clean_trans, str(src_freq), str(levdist), str(levdistfrac), str(aoa)]
                newlines.append(",".join(newrow)+'\n')

                
print(len(newlines))
with open(fnout,'w') as fp:
    fp.writelines(newlines)

Will save translations and features to:  fr_en_rootwordfeats.txt
le to the (LD=2, LDfrac=0.67)
garçon to boy (LD=5, LDfrac=0.83)
il to he (LD=2, LDfrac=1.00)
suivre to follow (LD=6, LDfrac=1.00)
un to a (LD=2, LDfrac=1.00)
femme to wife (LD=4, LDfrac=0.80)
fille to girl (LD=3, LDfrac=0.60)
riche to rich (LD=1, LDfrac=0.20)
rouge to red (LD=4, LDfrac=0.80)
homme to man (LD=4, LDfrac=0.80)
être to be (LD=3, LDfrac=0.75)
pomme to apple (LD=4, LDfrac=0.80)
taire to shut up (LD=7, LDfrac=1.00)
manger to eat (LD=5, LDfrac=0.83)
lui to him (LD=3, LDfrac=1.00)
calme to calm (LD=1, LDfrac=0.20)
avoir to have (LD=4, LDfrac=0.80)
enfant to child (LD=6, LDfrac=1.00)
et to and (LD=3, LDfrac=1.00)
robe to dress (LD=4, LDfrac=0.80)
chat to cat (LD=1, LDfrac=0.25)
noir to black (LD=5, LDfrac=1.00)
se to himself (LD=5, LDfrac=0.71)
sommer to summon (LD=3, LDfrac=0.50)
écrire to write (LD=3, LDfrac=0.50)
aimer to like (LD=3, LDfrac=0.60)
calmer to calm (LD=2, LDfrac=0.33)
journal to newspaper (LD=8, LDf

KeyboardInterrupt: 

In [37]:
# FOR EVERY WORD (NOT root word)
# Make csv of translation and calculate a simple metric of how much of a "cognate" it is

#output filename
fnout = "{}_{}_wordwordfeats.txt".format(src,dest)
print("Will save translations and features to: ",fnout)


trans_cache = {}

newlines=[]

with open(fn, 'r') as f:
    for line in f:
        if len(line)>1 and line[0]!='#':
            # process and translate
            pieces = line.split()
            idx = pieces[0]
            word = pieces[1]
            root_token = pieces[2]
            part_of_speech = pieces[3]
            morph_feats = pieces[4]
            dependency_label = pieces[5]
            dependency_edge_head = pieces[6]

            clean_word = word.lower() # Using WORD not root 
            if '\'' not in clean_word and clean_word not in trans_cache:
                # Wait so as not to annoy the API (otherwise will give JSON error)
                time.sleep(0.2)

                # Translate so we know the word in the language the user already knows
                translation_obj = translator.translate(clean_word,src=src,dest=dest)
                translation = translation_obj.text

                # Clean up translation: 
                # remove accents, make lowercase, and remove any non-informative first words
                clean_trans = unidecode(translation)
                clean_trans = clean_trans.lower() 
                g = clean_trans.split(" ")
                if len(g) == 2:
                    if g[0] in ['to','i','we','you','they','he','she','the']:
                        clean_trans = g[1]          
                elif len(g) > 2:
                    if g[1] in ['will']: # e.g. comere, i will eat
                        clean_trans = ' '.join(g[2:len(g)])

                # levenshtein distance features
                levdist = editdistance.eval(clean_word, clean_trans)
                levdistfrac = levdist/max(len(clean_word),len(clean_trans))
                
                if dest == 'en':
                    aoa_word = clean_trans
                else:
                    aoa_word = clean_word
                word_df = aoa_df[aoa_df['Word'] == aoa_word]
                if not word_df.empty:
                    aoa = word_df['Rating.Mean'].values[0]
                else:
                    aoa = np.nan
                    
                # print to console translation and LD 
                print("%s to %s (LD=%i, LDfrac=%.2f, AOA=%.2f)" % (clean_word, clean_trans, levdist, levdistfrac, aoa))
                trans_cache[clean_word]=translation

                # word frequency of translation (known language) and src words
                trans_freq = word_frequency(translation, dest)
                src_freq = word_frequency(word, src)

                # write to csv file
                newrow = [word, clean_trans, str(src_freq), str(levdist), str(levdistfrac), str(aoa)]
                newlines.append(",".join(newrow)+'\n')

                
print(len(newlines))
with open(fnout,'w') as fp:
    fp.writelines(newlines)

Will save translations and features to:  fr_en_wordwordfeats.txt
le to the (LD=2, LDfrac=0.67, AOA=3.98)
garçon to boy (LD=5, LDfrac=0.83, AOA=3.67)
je to i (LD=2, LDfrac=1.00, AOA=nan)
suis to am (LD=4, LDfrac=1.00, AOA=nan)
une to a (LD=3, LDfrac=1.00, AOA=2.89)
femme to wife (LD=4, LDfrac=0.80, AOA=5.67)
la to the (LD=3, LDfrac=1.00, AOA=3.98)
fille to girl (LD=3, LDfrac=0.60, AOA=4.00)
un to a (LD=2, LDfrac=1.00, AOA=2.89)
riche to rich (LD=1, LDfrac=0.20, AOA=6.32)
rouge to red (LD=4, LDfrac=0.80, AOA=3.68)
homme to man (LD=4, LDfrac=0.80, AOA=3.11)
est to east (LD=1, LDfrac=0.25, AOA=6.94)
pomme to apple (LD=4, LDfrac=0.80, AOA=4.15)
tu to you (LD=2, LDfrac=0.67, AOA=4.35)
es to es (LD=0, LDfrac=0.00, AOA=nan)
mange to eat (LD=4, LDfrac=0.80, AOA=2.78)
il to he (LD=2, LDfrac=1.00, AOA=3.81)
elle to she (LD=3, LDfrac=0.75, AOA=3.57)
calme to calm (LD=1, LDfrac=0.20, AOA=6.26)
a to at (LD=1, LDfrac=0.50, AOA=4.04)
enfant to child (LD=6, LDfrac=1.00, AOA=5.15)
et to and (LD=3, LDfra

In [44]:
with open(fnout,'w') as fp:
    fp.writelines(newlines)

In [45]:
fnout

'fr_en_wordwordfeats.txt'

In [19]:
print("donedone")

donedone


In [14]:
clean_trans = "i will eat"
g = clean_trans.split(" ")
if len(g) == 2:
    if g[0] in ['to','i','we','you','they','he','she','the',"i'll"]:
        clean_trans = g[1]          
elif len(g) > 2:
    if g[1] in ['will']:
        clean_trans = ' '.join(g[2:len(g)])
    print("string with %i parts detected" % (len(g)))
    
print(clean_trans)

string with 3 parts detected
eat


In [21]:
xl = pd.ExcelFile("data/13428_2013_348_MOESM1_ESM.xlsx")
# Print the sheet names
print(xl.sheet_names)

# Load a sheet into a DataFrame by name: df1
df1 = xl.parse('Sheet1')

['Sheet1', 'Sheet2', 'Sheet3']


In [25]:
train_x

NameError: name 'train_x' is not defined

In [24]:
df1['Word']

0                   a
1            aardvark
2              abacus
3             abalone
4             abandon
5           abandoner
6         abandonment
7               abase
8           abasement
9               abate
10          abatement
11           abattoir
12             abbacy
13             abbess
14              abbey
15              abbot
16         abbreviate
17        abbreviated
18       abbreviation
19           abdicate
20         abdication
21            abdomen
22          abdominal
23             abduct
24          abduction
25           abductor
26              abeam
27           aberrant
28         aberration
29               abet
             ...     
31094         zillion
31095       zillionth
31096            zinc
31097       zinfandel
31098            zing
31099          zinger
31100             zip
31101          zipper
31102        zippered
31103           zippy
31104          zircon
31105       zirconium
31106             zit
31107          zither
31108     